In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

## Read Data

In [2]:
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num','marital-status', 'occupation', 'relationship', 'race', 'gender','capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']

train = pd.read_csv('./Census Income Data Set/Census Income Data Set/adult.data.txt', sep=",\s", header=None, names = column_names, engine = 'python')
test = pd.read_csv('./Census Income Data Set/Census Income Data Set/adult.test.txt', sep=",\s", header=None, names = column_names, engine = 'python')
test['income'].replace(regex=True,inplace=True,to_replace=r'\.',value=r'')

## Data Preparation

In [3]:
train.replace('?', pd.NA, inplace=True)
train_cleaned = train.dropna(axis=0)

test.replace('?', pd.NA, inplace=True)
test_cleaned = test.dropna(axis=0)

In [4]:
# Setting all the categorical columns to type category
for col in set(train_cleaned.columns) - set(train_cleaned.describe().columns):
    train_cleaned[col] = train_cleaned[col].astype('category')
print(train_cleaned.info())

# Setting all the categorical columns to type category
for col in set(test_cleaned.columns) - set(test_cleaned.describe().columns):
    test_cleaned[col] = test_cleaned[col].astype('category')
print(test_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   age              30162 non-null  int64   
 1   workclass        30162 non-null  category
 2   fnlwgt           30162 non-null  int64   
 3   education        30162 non-null  category
 4   educational-num  30162 non-null  int64   
 5   marital-status   30162 non-null  category
 6   occupation       30162 non-null  category
 7   relationship     30162 non-null  category
 8   race             30162 non-null  category
 9   gender           30162 non-null  category
 10  capital-gain     30162 non-null  int64   
 11  capital-loss     30162 non-null  int64   
 12  hours-per-week   30162 non-null  int64   
 13  native-country   30162 non-null  category
 14  income           30162 non-null  category
dtypes: category(9), int64(6)
memory usage: 1.9 MB
None
<class 'pandas.core.frame.DataFrame'

C:\Users\86139\AppData\Local\Temp\ipykernel_3464\336448181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_cleaned[col] = train_cleaned[col].astype('category')
C:\Users\86139\AppData\Local\Temp\ipykernel_3464\336448181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_cleaned[col] = train_cleaned[col].astype('category')
C:\Users\86139\AppData\Local\Temp\ipykernel_3464\336448181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [5]:
# Summary
train_cleaned.describe()
test_cleaned.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,15060.000000,1.506000e+04,15060.000000,15060.000000,15060.000000,15060.000000
mean,38.768327,1.896164e+05,10.112749,1120.301594,89.041899,40.951594
std,13.380676,1.056150e+05,2.558727,7703.181842,406.283245,12.062831
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.166550e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.779550e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.385888e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000


In [6]:
train_data = train_cleaned.drop(columns = ['income', 'fnlwgt'])
train_label = train_cleaned['income']

test_data = test_cleaned.drop(columns = ['income', 'fnlwgt'])
test_label = test_cleaned['income']

In [7]:
## Data Preprocessing

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
numerical_cols = ['age', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

categorical_transformer = OneHotEncoder()

numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

train_data_transformed = pipeline.fit_transform(train_data)
test_data_transformed = pipeline.transform(test_data)

In [8]:
train_data_transformed_dense = train_data_transformed.toarray()
test_data_transformed_dense = test_data_transformed.toarray()

In [9]:
label_mapping = {'<=50K': 0, '>50K': 1}
train_label = train_label.map(label_mapping)
test_label = test_label.map(label_mapping)

# Decision Tree

In [65]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=None, min_samples_leaf=None, alpha = None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.alpha = alpha
        self.tree = None

    # Calculating Entropy
    def entropy(self, y):
        ent = 0.0
        unique, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        ent = -np.sum(probabilities * np.log2(probabilities))
        return ent

    # Splitting Data
    def split(self, X, y, feature, threshold):
        left_mask = X[:, feature] <= threshold
        right_mask = X[:, feature] > threshold
        return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

    # Calculating Information Gain
    def information_gain(self, X, y, feature, threshold):
        parent_entropy = self.entropy(y)
        X_left, X_right, y_left, y_right = self.split(X, y, feature, threshold)
        # if any child node is null, information gain is 0
        if len(y_left) == 0 or len(y_right) == 0:
            return 0
        # weighted entropy
        n = len(y)
        n_left, n_right = len(y_left), len(y_right)
        weighted_entropy = (n_left / n) * self.entropy(y_left) + (n_right / n) * self.entropy(y_right)
        information_gaining = parent_entropy - weighted_entropy
        return information_gaining

    # Best Split
    def best_split(self, X, y):
        # initialization
        best_gain = -1
        best_feature = None
        best_threshold = None
        # traversal all features
        for feature in range(X.shape[1]):
            unique_values = np.unique(X[:, feature])
            # one feature -> skip
            if len(unique_values) == 1:
                continue
            # traversal all thresholds
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self.information_gain(X, y, feature, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

    # Building Tree
    def build_tree(self, X, y, depth=0):
        # stop condictions
        # prediction result
        if len(np.unique(y)) == 1:
            return y[0]
        # max depth
        if self.max_depth is not None and depth >= self.max_depth:
            return np.bincount(y).argmax()
        # min split samples
        if len(y) < self.min_samples_split:
            return np.bincount(y).argmax()
        # best split and shreshold
        feature, threshold = self.best_split(X, y)
        if feature is None:
            return np.bincount(y).argmax()
        # split data
        X_left, X_right, y_left, y_right = self.split(X, y, feature, threshold)
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return np.bincount(y).argmax()
        # build the subtree recursively
        left_subtree = self.build_tree(X_left, y_left, depth + 1)
        right_subtree = self.build_tree(X_right, y_right, depth + 1)
        return {'feature': feature, 'threshold': threshold, 'left': left_subtree, 'right': right_subtree}

    def fit(self, X, y):
        self.tree = self.build_tree(np.array(X), np.array(y))
        self.tree = self.prune_tree(self.tree, np.array(X), np.array(y))

    # Prediction
    def predict_sample(self, x, tree):
        if isinstance(tree, dict):
            feature = tree['feature']
            threshold = tree['threshold']
            if x[feature] <= threshold:
                return self.predict_sample(x, tree['left'])
            else:
                return self.predict_sample(x, tree['right'])
        else:
            return tree

    def predict(self, X):
        return [self.predict_sample(x, self.tree) for x in X]
    
    def prune_tree(self, tree, X, y):
        # Recursively prune the left and right subtrees
        if isinstance(tree, dict):
            tree['left'] = self.prune_tree(tree['left'], X, y)
            tree['right'] = self.prune_tree(tree['right'], X, y)
            # If both left and right subtrees are leaves (not dicts), consider pruning
            if not isinstance(tree['left'], dict) and not isinstance(tree['right'], dict):
                # Evaluate the cost complexity of pruning this subtree
                original_accuracy = self.evaluate_tree(tree, X, y)
                pruned_tree = np.bincount(y).argmax()
                pruned_accuracy = self.evaluate_tree(pruned_tree, X, y)
                if pruned_accuracy - original_accuracy < self.alpha:
                    # Prune the tree
                    return pruned_tree
        return tree

    def evaluate_tree(self, tree, X, y):
        predictions = [self.predict_sample(x, tree) for x in X]
        return np.mean(predictions == y)

In [74]:
def model_eval(actual, pred):
    
    confusion = pd.crosstab(actual, pred, rownames=['Actual'], colnames=['Predicted'])
    TP = confusion.loc[1,1]
    TN = confusion.loc[0,0]
    FP = confusion.loc[0,1]
    FN = confusion.loc[1,0]

    accuracy = ((TP+TN))/(TP+FN+FP+TN)
    precision = (TP)/(TP+FP)
    recall = (TP)/(TP+FN)
    f1_score = (2*recall*precision)/(recall+precision)
    
    out = {}
    out['accuracy'] =  accuracy
    out['precision'] = precision
    out['recall'] = recall
    out['f1_score'] = f1_score
    return out

In [68]:
tree = DecisionTree(max_depth=10, min_samples_split=2, min_samples_leaf=1, alpha = 0.001)
tree.fit(train_data_transformed_dense, train_label)
predictions = tree.predict(test_data_transformed_dense)
acc = np.sum(predictions == test_label) / len(test_label)
print(f"Accuracy = ", acc)

Accuracy =  0.8542496679946879


In [75]:
output = model_eval(test_label, predictions)
print(output)

{'accuracy': 0.8542496679946879, 'precision': 0.783427495291902, 'recall': 0.5621621621621622, 'f1_score': 0.6546026750590086}


In [76]:
from graphviz import Digraph

def visualize_tree(node, dot=None):
    if dot is None:
        dot = Digraph(comment='Decision Tree')

    if isinstance(node, dict):
        feature = node['feature']
        threshold = node['threshold']
        label = f'Feature {feature} <= {threshold}'
        dot.node(str(id(node)), label=label)
        
        if 'left' in node:
            left_child = node['left']
            dot.edge(str(id(node)), str(id(left_child)), label='True')
            visualize_tree(left_child, dot)
        
        if 'right' in node:
            right_child = node['right']
            dot.edge(str(id(node)), str(id(right_child)), label='False')
            visualize_tree(right_child, dot)
    else:
        dot.node(str(id(node)), label=f'Value: {node}', shape='ellipse')
    
    return dot

dot = visualize_tree(tree.tree)

dot.render('decision_tree', format='png', view=True)

'decision_tree.png'

In [27]:
## Just For Comparison
## from sklearn.tree import DecisionTreeClassifier
## tree_compare = DecisionTreeClassifier()
## tree_compare.fit(train_data_transformed_dense, train_label)
## tree_compare.score(test_data_transformed_dense, test_label)

0.8116865869853918